<a href="https://colab.research.google.com/github/JasperLS/data_science_intros/blob/main/LSTM_Example_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
from google.colab import drive
drive.mount('/content/drive')

# **Vorbereitung Daten** (Preprocessing)

In [ ]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.utils import shuffle

In [ ]:
# Randomize Index to get shuffled arrays 
array = list(range(lstm_ivs.shape[0]))
new = shuffle(array, random_state=142)

df_ivs = pd.read_csv('/content/drive/My Drive/computer vision intro/data_final_ivs.csv', sep=';').reindex(new)
df_dvs = pd.read_csv('/content/drive/My Drive/computer vision intro/data_final_dvs.csv', sep=';').reindex(new)
lstm_ivs = np.load('/content/drive/My Drive/computer vision intro/lstm_all_ivs.npy')
lstm_ivs = lstm_ivs[new,:,:]

In [ ]:
# Delete some columns
del df_ivs['video_id']
del df_ivs['number_of_scenes']
del df_ivs['number_of_faces']
del df_ivs['avg_length_of_scenes']
del df_ivs['length_vid']
del df_dvs['video_id']
del df_dvs['kunde']

In [ ]:
#Berücksichtigung der Dummy Variable "Kunde" im DataFrame der unabhängigen Variablen
dummy_var = pd.get_dummies(df_ivs['kunde'], prefix='Kunde', drop_first=True)
df = pd.concat([df_ivs, dummy_var], axis = 1)
pd.set_option('display.max_columns', None)
#separate Dummy for LSTM
df_dummys = df[['Kunde_1','Kunde_2','Kunde_3','Kunde_4','Kunde_5','Kunde_6','Kunde_7','Kunde_8','Kunde_9','Kunde_10','Kunde_11','Kunde_13','Kunde_14','Kunde_15','Kunde_16','Kunde_17','Kunde_18','Kunde_19','Kunde_20']]

In [ ]:
# Delete some columns
del df_ivs['kunde']
del df['kunde']

In [ ]:
# Select DV
df_dvs = df_dvs[['ctr']]*100 

In [ ]:
# Select IVs
df = df[['colorfulness_avg','saturation_avg', 'value_avg', 'black_avg', 'blue_avg', 'brown_avg', 'grey_avg','green_avg', 'orange_avg','pink_avg','purple_avg','red_avg','yellow_avg','white_avg','avg_scene_freq','Kunde_1','Kunde_2','Kunde_3','Kunde_4','Kunde_5','Kunde_6','Kunde_7','Kunde_8','Kunde_9','Kunde_10','Kunde_11','Kunde_13','Kunde_14','Kunde_15','Kunde_16','Kunde_17','Kunde_18','Kunde_19','Kunde_20']]

In [ ]:
#LSTM auf entsprechende IV(s) beschränken
lstm_ivs = lstm_ivs[:,:,:]

In [ ]:
df.head(2)
df_dvs.head(2)
df.shape
df_dvs.shape
lstm_ivs.shape

In [ ]:
'''

What's the third dimension of LSTM IVs?

'''

### Training- und Test-Set

In [ ]:
#Unterteilung in Training- und Test-Set
split_size=2487 #80% Train- und 20% Test-Set
#Regression
y_train_reg = df_dvs[:split_size]
y_test_reg = df_dvs[split_size:]
X_train_reg = df[:split_size]
X_test_reg = df[split_size:]
#LSTM
y_train_lstm = df_dvs[:split_size]
y_test_lstm = df_dvs[split_size:]
dummys_train_lstm = df_dummys[:split_size]
dummys_test_lstm = df_dummys[split_size:]
iv_lstm_train = lstm_ivs[:split_size,:,:]
iv_lstm_test = lstm_ivs[split_size:,:,:]

# **Regression**

In [ ]:
%matplotlib inline

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
'''

What problem do we probably encounter?

'''

## SKIP: Plots

In [ ]:
#Plot
# plt.figure(figsize=(20,7))

# y1 = df_dvs
# y2 = df_dvs
# x1 = df[['colorfulness_avg']]
# x2 = df[['avg_scene_freq']]

# plt.subplot(121)
# plt.scatter(x1, y1)
# plt.xlim(0.0, 1.0)
# plt.ylim(0.0, 2.0)
# plt.ylabel('Click-Through-Rate', fontsize=14)
# plt.xlabel('Colorfulness', fontsize=14)

# plt.subplot(122)
# plt.scatter(x2, y2)
# plt.xlim(0.0, 1.0)
# plt.ylim(0.0, 2.0)
# plt.ylabel('Click-Through-Rate', fontsize=14)
# plt.xlabel('Szenen-Frequenz', fontsize=14)

## Modell

In [ ]:
X = df
Y = df_dvs

#X = sm.add_constant(X)

model = sm.OLS(Y.astype(float), X.astype(float))
results = model.fit()

print(results.summary())



In [ ]:
# plt.rc('figure', figsize=(12, 7))
# plt.text(0.01, 0.05, str(results.summary()), {'fontsize': 10}, fontproperties = 'monospace') 
# plt.axis('off')
# plt.tight_layout()

## VIF Check

In [ ]:
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

print(vif_data)

In [ ]:
plt.rc('figure', figsize=(12, 7))
plt.text(0.01, 0.05, str(vif_data), {'fontsize': 10}, fontproperties = 'monospace') 
plt.axis('off')
plt.tight_layout()

## Training


In [ ]:
# Repeat training for train set only for later comparison
lin_reg_mod = LinearRegression()
lin_reg_mod.fit(X_train_reg, y_train_reg)

## Test & Prediction

In [ ]:
y_pred_reg = lin_reg_mod.predict(X_test_reg)

df_compare_reg = pd.DataFrame({'Tatsächlicher Wert': y_test_reg.values.flatten(), 'Vorhersage': y_pred_reg.flatten()})
df_compare_reg

In [ ]:
# df1 = df_compare_reg.head(50)
# df1.plot(kind='bar',figsize=(16,10))
# plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
# plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# plt.show()

## MSE

In [ ]:
#Berechnung MSE
reg_mse = metrics.mean_squared_error(y_test_reg, y_pred_reg)
#Rücktransformation des MSE
reg_mse = reg_mse/10000

print('Die Regression hat einen MSE von', reg_mse)

# **Long Short-Term Memory (LSTM)**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Input, Dense, Dropout, Concatenate
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras import optimizers

pd.options.display.float_format = "{:,.5f}".format

In [ ]:
'''

Before we continue:
What are potential tuning parameters

'''

## Modell

In [ ]:
## Parameter LSTM
lstm_shape = 10 
main_shape = 24 # [8,16,24,30]
learn = 0.3 # [0.1,0.2,0.3,0.4]
epochs = 150 # [50,100,150,200]
batch_size = 64 # [16,32,64,128]
drop = 0.3 # [0.1,0.2,0.3,0.4]
dense = 12
opt = optimizers.Adadelta(learning_rate=learn) # [Adadelta,Adam,Adamax,Adagrad,SGD]

In [ ]:
#Modellaufbau / -struktur
first_input = Input(shape=(lstm_ivs.shape[1:]), name='first_input')
lstm_out = LSTM(lstm_shape,dropout= drop,return_sequences = False,return_state=False)(first_input)
first = Dense(dense)(lstm_out)
first = Dropout(drop)(first)

second_input = Input(shape = df_dummys.shape[1], name='dummys')
second = Dense(main_shape,activation = 'relu')(second_input)

l = Concatenate(axis=1)([first,second])
l = Dense(main_shape, activation='relu')(l)
l = Dropout(drop)(l)
l = Dense(main_shape, activation='relu')(l)
l = Dropout(drop)(l)

main_output = Dense(1,activation = 'relu', name='main_output')(l)

model = Model(inputs=[first_input,second_input], outputs=[main_output]) 
model.compile(optimizer = opt, loss='mean_squared_error')
model.reset_states() 
model.summary()

In [ ]:
plot_model(model,show_shapes=True, show_layer_names=True)

## Training

In [ ]:
history = model.fit([iv_lstm_train, dummys_train_lstm], y_train_lstm, epochs=epochs, batch_size=batch_size , use_multiprocessing = True, validation_data=([iv_lstm_test, dummys_test_lstm], y_test_lstm))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Test & Prediction


In [ ]:
#Prediction
# y_pred_lstm = model.predict([iv_lstm_test, dummys_test_lstm])
# df_compare_lstm = pd.DataFrame({'Tatsächlicher Wert': y_test_lstm.values.flatten(), 'Vorhersage': y_pred_lstm.flatten()})
# df_compare_lstm

## MSE

In [ ]:
#Berechnung MSE
lstm_mse = metrics.mean_squared_error(y_test_lstm, y_pred_lstm)
#Rücktransformation des MSE
lstm_mse = lstm_mse/10000

print('Das LSTM hat einen MSE von', lstm_mse)

# Baseline Modell

In [ ]:
'''

What could we use as an alternative baseline model?

'''

## Training

In [ ]:
#Test-Set Baseline
y_test_base = y_test_reg.copy()

In [ ]:
#Prediction-Set Baseline
y_pred_base = y_test_reg.copy()

In [ ]:
#Durchschnittswert aus Trainings-Set als Predictor
y_pred_base.insert(1,'ctr_avg',y_train_reg['ctr'].mean())

In [ ]:
#Zwischenschritt für Berechnung der Abweichung der Vorhersage
df_compare_base = y_pred_base.copy()

In [ ]:
#Dataframe bereinigen
del y_pred_base['ctr']

## MSE

In [ ]:
#Berechnung MSE
base_mse = metrics.mean_squared_error(y_test_base, y_pred_base)
#Rücktransformation des MSE
base_mse = base_mse/10000

print('Das Baseline Modell hat einen MSE von', base_mse)

# **Comparison: Baseline vs. Regression vs. LSTM**

In [ ]:
modelle = ['Baseline','Regression', 'LSTM']
werte = [base_mse, reg_mse, lstm_mse]

plt.figure(figsize=(22, 8))

plt.subplot(131)

plt.bar(modelle, werte)
plt.title('Vergleich: Baseline vs. Regression vs. LSTM', fontsize=16)

In [ ]:
#Prozentuale Unterschiede
reg_mse_copy = reg_mse
lstm_mse_copy = lstm_mse
base_mse_copy = base_mse

diff_base_reg = (1-reg_mse_copy/base_mse_copy)*100
diff_base_lstm = (1-lstm_mse_copy/base_mse_copy)*100
diff_reg_lstm = (1-lstm_mse_copy/reg_mse_copy)*100

print('Das Baseline Modell hat einen MSE von',"{:.7f}".format(base_mse))
print('Die Regression hat einen MSE von',"{:.7f}".format(reg_mse),'und ist damit',"{:.2f}".format(diff_base_reg),'Prozent besser als das Baseline-Modell.')
print('Das LSTM hat einen MSE von',"{:.7f}".format(lstm_mse),'und ist damit',"{:.2f}".format(diff_base_lstm),'Prozent besser als das Baseline-Modell sowie',"{:.2f}".format(diff_reg_lstm),'Prozent besser als die Regression.')